In [62]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import random
from datetime import datetime
import json
from postgre_cred import database_url
from hashlib import md5

In [9]:
engine = create_engine(database_url)

In [10]:
def generate_url(full_slug: str, cat_id: int):
    url = f'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/{full_slug}/{cat_id}.json?'
    slug_parts = full_slug.split('/')
    slug_parts = list(map(lambda x: f'type={x}&', slug_parts))
    slug_parts = ''.join(slug_parts)
    url = f'{url}{slug_parts}type={cat_id}'
    return url

In [11]:
from functools import cache

@cache
def get_all_sub_cat_dataframes(url):
    tech_sub_cat_data = requests.get(url).json()
    
    # If it's a leaf category with no subcategories
    if len(tech_sub_cat_data['pageProps']['data']['categories']) == 0:
        print(f"{tech_sub_cat_data['pageProps']['data']['headline']} IS A LEAF CATEGORY!")
        return [extract_data(url)]  # Return as a list containing this df
    else:
        tech_sub_cat_data = tech_sub_cat_data['pageProps']['data']['categories']
        tech_sub_cat_df = pd.DataFrame(tech_sub_cat_data)
        tech_sub_cat_df = tech_sub_cat_df[['id', 'full_slug']]
        
        urls = tech_sub_cat_df.apply(lambda row: generate_url(row['full_slug'], row['id']), axis=1)
        
        result = []
        for sub_url in urls:
            result.extend(get_all_sub_cat_dataframes(sub_url))
        return result

In [12]:
def extract_data(url):
    page = 1
    products = []
    slug = 'tmp'
    while True:
        resp = requests.get(f'{url}&page_size=1000&page={page}')
        resp = resp.json()
        if page == 1:
            slug = resp['pageProps']['data']['fullSlug']
            slug = slug.lower().replace('-', '_').replace(' ', '-').replace('/', '_')
        current_products = resp['pageProps']['data']['products']
        if not len(current_products):
            break
        products.extend(current_products)
        page += 1
    
    
    df = pd.DataFrame(products)
    print(slug, df.shape, sep='  ->  ')
    print('DATA EXTRACTION SUCCESS!')
    return [df, slug]

In [45]:
def clean_data(df: pd.DataFrame):
    try:
        df['price'] = df['stock'].apply(lambda x: x['price'])
        df['old_price'] = df['stock'].apply(lambda x: x['start_price'])
        df = df[['price', 'old_price', 'headline', 'image', 'slug']].copy() 
        df = df.rename(columns={'headline': 'name', 'slug': 'url'}) 
        print('DATA CLEANING SUCCESS!')
        return df
    except Exception as e:
        print(f'ERROR {e}')
        

In [55]:
def export_data(df: pd.DataFrame, db_name):
    try:
        df.to_sql(db_name, con=engine, if_exists='replace', index=True)
        print(f'{db_name} DATA EXPORT SUCCESS!')
    except Exception as e:
        print(f'ERROR ON {db_name}\n {e}\n')

In [17]:
import pickle

with open('dfs_and_names.pkl', 'wb') as f:
    pickle.dump(dfs_n_names, f)

In [84]:
with open("dfs_and_names.pkl", "rb") as f:
    dfs_n_names = pickle.load(f)

In [93]:
def main():
    with open("dfs_and_names.pkl", "rb") as f:
        dfs_n_names = pickle.load(f)
    # main_url = 'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/teqnika/774.json?type=teqnika&type=774'
    # dfs_n_names = get_all_sub_cat_dataframes(main_url)
    dfs_n_names = list(filter(lambda c: c[0].shape != (0, 0), dfs_n_names))
    dfs_n_names = list(map(lambda x: [clean_data(x[0]), x[1]], dfs_n_names))
    dfs_n_names = list(map(lambda x: [x[0], f'veli_{md5(x[1].encode()).hexdigest()}'], dfs_n_names))
    finish = list(map(lambda c: export_data(c[0], c[1]), dfs_n_names))

In [94]:
main()

DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANING SUCCESS!
DATA CLEANI